In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import h5py
import io

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, TargetEncoder

import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from tqdm.notebook import tqdm

In [2]:
# Configs
RANDOM_SEED = 42
N_FOLDS = 5
maj_frac = 0.1
min_frac = 10

In [3]:
# set random seed
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [4]:
# Check if GPU is available
print("GPU", "available (YES!)" if tf.config.list_physical_devices('GPU') else "not available :(")

# If GPU is available, use it
if tf.config.list_physical_devices('GPU'):
    device = '/GPU:0'
else:
    device = '/CPU:0'

print(device)

GPU not available :(
/CPU:0


In [5]:
# load data
metadata_df = pd.read_csv('train_processed.csv')
train_val_img = h5py.File(f'train-image.hdf5', 'r')

In [9]:
# Define the Data Generator
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

class TestDataLoader:
    def __init__(self, hdf5_file, metadata_df, batch_size=32):
        self.hdf5_file = hdf5_file
        self.metadata_df = metadata_df.reset_index(drop=True)
        self.batch_size = batch_size

    def __len__(self):
        # Number of full batches
        return (len(self.metadata_df) + self.batch_size - 1) // self.batch_size

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = start_idx + self.batch_size
        # If the end index exceeds the data length, adjust it for the last batch
        batch_indices = range(start_idx, min(end_idx, len(self.metadata_df)))
        
        images = []
        for index in batch_indices:
            isic_id = self.metadata_df.iloc[index]['isic_id']
            img = self.load_image(isic_id)
            images.append(img)

        return np.array(images)

    def load_image(self, isic_id):
        byte_string = self.hdf5_file[isic_id][()]
        nparr = np.frombuffer(byte_string, np.uint8)
        img = Image.open(io.BytesIO(nparr))
        img = img.resize((128, 128))
        img = preprocess_input(np.array(img))
        return img

    def __iter__(self):
        for i in range(len(self)):
            yield self[i]

In [10]:
# Define custom metric
from sklearn.metrics import roc_auc_score

def partial_auc_metric(y_true, y_pred):
    def compute_partial_auc(y_true, y_pred):
        # Convert tensors to numpy arrays
        y_true_np = y_true.numpy()
        y_pred_np = y_pred.numpy()

        # Compute partial AUC as per your custom function
        min_tpr = 0.80
        max_fpr = abs(1 - min_tpr)
        
        v_gt = abs(y_true_np - 1)
        v_pred = np.array([1.0 - x for x in y_pred_np])
        
        partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
        partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
        
        return partial_auc

    # Wrap the custom Python function with tf.py_function
    result = tf.py_function(func=compute_partial_auc, inp=[y_true, y_pred], Tout=tf.float32)
    
    return result

In [15]:
# Load the models
from tensorflow.keras.models import load_model

# Load models with custom metric
model_paths = ['Fold1/model_11.h5', 'Fold2/model_03.h5', 'Fold3/model_06.h5', 'Fold4/model_09.h5', 'Fold5/model_02.h5']
models = [load_model(path, custom_objects={'partial_auc_metric': partial_auc_metric}) for path in model_paths]

In [79]:
print(tf.__version__)

2.12.0


In [80]:
# Save each model using SavedModel format
for i, model in enumerate(models):
    model_json = model.to_json()
    with open(f"model_{i+1}.json", "w") as json_file:
        json_file.write(model_json)
    
    model.save_weights(f"model_{i+1}.h5")
    

In [19]:
# Define a function to make predictions to avoid retracing
@tf.function
def make_predictions(model, batch):
    return model(batch, training=False)

In [20]:
# Define data loader
test_loader = TestDataLoader(train_val_img, metadata_df)

test_dataset = tf.data.Dataset.from_generator(
    lambda: test_loader,
    output_signature=(
        tf.TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32)
    )
)

# Estimate the number of batches in the test dataset
num_batches = len(test_loader)

# Initialize prediction dictionary
predictions = {f'M{i+1}': [] for i in range(len(models))}

for batch in tqdm(test_dataset, total=num_batches, desc='Predicting'):
    for i, model in enumerate(models):
        predictions[f'M{i+1}'].append(make_predictions(model, batch))

# Convert predictions to numpy arrays
for key in predictions.keys():
    predictions[key] = np.concatenate([p.numpy() for p in predictions[key]], axis=0)

Predicting:   0%|          | 0/12534 [00:00<?, ?it/s]

2024-11-12 13:56:50.338216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [28]:
# Calculate pAUC for each model
paucs = []

for key in predictions.keys():
    pauc = partial_auc_metric(metadata_df['target'], predictions[key])
    paucs.append(pauc)

# Calculate the mean pAUC
mean_pauc = np.mean(paucs)
print(f'Mean pAUC: {mean_pauc}')

Mean pAUC: 0.13719458878040314


In [22]:
for key in predictions.keys():
    print(f'{key}: {predictions[key].shape}')

M1: (401059, 1)
M2: (401059, 1)
M3: (401059, 1)
M4: (401059, 1)
M5: (401059, 1)


In [24]:
# Stack predictions from all models along a new axis and calculate the mean across this axis
# This will result in an array of shape (401059, 1)
cnn_confidence = np.mean(
    np.stack([predictions['M1'], predictions['M2'], predictions['M3'], predictions['M4'], predictions['M5']], axis=1),
    axis=1
)

In [26]:
# Add the cnn_confidence as a new column in metadata_df
metadata_df['cnn_confidence'] = cnn_confidence

In [29]:
metadata_df['cnn_confidence'].describe()

count    401059.000000
mean          0.196883
std           0.214336
min           0.000030
25%           0.031529
50%           0.113297
75%           0.296966
max           0.998634
Name: cnn_confidence, dtype: float64

In [30]:
# Add noise to the CNN confidence
noise_level = 0.015

metadata_df['cnn_confidence'] = metadata_df['cnn_confidence'] + np.random.normal(0, noise_level, metadata_df.shape[0])
metadata_df['cnn_confidence'] = metadata_df['cnn_confidence'].clip(0, 1)

In [32]:
# Add folds using StratifiedGroupKFold
metadata_df['fold'] = -1
sgkf = StratifiedGroupKFold(n_splits=N_FOLDS)

for fold, (train_val_idx, test_idx) in enumerate(sgkf.split(metadata_df, metadata_df['target'], metadata_df['patient_id'])):
    metadata_df.loc[test_idx, 'fold'] = fold

In [48]:
# Binary encoding for 'sex' column
le = LabelEncoder()
metadata_df['sex_encoded'] = le.fit_transform(metadata_df['sex'])

ordinal_encoder = OrdinalEncoder()
metadata_df['age_group'] = ordinal_encoder.fit_transform(metadata_df['age_group'].values.reshape(-1, 1))

X = metadata_df[['tbp_lv_location_simple']]
y = metadata_df['target']

# Initialize the TargetEncoder with smoothing (adjust `smooth` and `cv` as needed)
encoder = TargetEncoder(smooth='auto', cv=5, shuffle=True, random_state=42)

# Fit and transform the data
X_encoded = encoder.fit_transform(X, y)

# Add the encoded feature back to the DataFrame
metadata_df['tbp_location_encoded'] = X_encoded

In [51]:
cols_to_drop = ['isic_id', 'patient_id', 'target', 'fold', 'sex', 'tbp_lv_location_simple']
train_cols = [col for col in metadata_df.columns if col not in cols_to_drop]

In [33]:
def pAUC(y_true, y_pred):
    # Compute partial AUC as per your custom function
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_pred])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

# Define the custom metric
def custom_lgbm_metric(y_true, y_pred):
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)

    # Convert y_true to 0/1 format for binary classification
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_pred])

    # Calculate partial AUC
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    # Return the metric name, value, and maximization indicator
    return 'PAUC', partial_auc, True

# LGBM Tree

In [63]:
# Define objective function
def lgbm_objective(trial):
    # Define hyperparameters
    param = {
        "objective": "binary",
        "metric": "custom",
        "verbosity": -1,
        "boosting_type": "gbdt",
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "device": "cpu",
        'random state': RANDOM_SEED
    }

    # Initialize list to store AUC scores
    aucs = []

    # Iterate over folds
    for fold in range(N_FOLDS):
        # Get train and validation data
        train_data = metadata_df[metadata_df['fold'] != fold].reset_index(drop=True)
        val_data = metadata_df[metadata_df['fold'] == fold].reset_index(drop=True)

        # Define features and target
        features = train_data[train_cols]
        target = train_data['target']

        dtrain = lgb.Dataset(features, target)

        # Train model
        model = lgb.train(param, dtrain,)
        preds = model.predict(val_data[train_cols])
        auc = pAUC(val_data['target'], preds)
        aucs.append(auc)

    return np.mean(aucs)

In [64]:
# Define study
study = optuna.create_study(direction="maximize", study_name='lgbm_tuning')
study.optimize(lgbm_objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
lgbm_trial = study.best_trial

print("  Value: {}".format(lgbm_trial.value))

print("  Params: ")
for key, value in lgbm_trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-11-12 16:26:15,444] A new study created in memory with name: lgbm_tuning


[I 2024-11-12 16:26:42,431] Trial 0 finished with value: 0.16240818240489427 and parameters: {'learning_rate': 0.004374159873139222, 'lambda_l1': 0.05311456051183232, 'lambda_l2': 0.0032222805283885917, 'num_leaves': 178, 'feature_fraction': 0.5323838651412293, 'bagging_fraction': 0.8554252074635839, 'bagging_freq': 2, 'min_child_samples': 83}. Best is trial 0 with value: 0.16240818240489427.
[I 2024-11-12 16:26:58,789] Trial 1 finished with value: 0.16712554894013065 and parameters: {'learning_rate': 0.013071429501476318, 'lambda_l1': 1.9873439946964387e-06, 'lambda_l2': 0.012795509170231572, 'num_leaves': 252, 'feature_fraction': 0.8074594807870954, 'bagging_fraction': 0.8191371292001413, 'bagging_freq': 4, 'min_child_samples': 100}. Best is trial 1 with value: 0.16712554894013065.
[I 2024-11-12 16:27:13,111] Trial 2 finished with value: 0.1584134751994358 and parameters: {'learning_rate': 0.0006108119884128023, 'lambda_l1': 2.4526872253941352e-05, 'lambda_l2': 0.023008593108168527, 

Number of finished trials: 20
Best trial:
  Value: 0.17495870069767475
  Params: 
    learning_rate: 0.03947661966596745
    lambda_l1: 6.542171607127298
    lambda_l2: 0.7407506117943363
    num_leaves: 20
    feature_fraction: 0.7611166336769638
    bagging_fraction: 0.8202373097760443
    bagging_freq: 1
    min_child_samples: 63


In [66]:
# Use the best hyperparameters
best_params = lgbm_trial.params
best_params['objective'] = 'binary'
best_params['metric'] = 'custom'
best_params['verbosity'] = -1
best_params['boosting_type'] = 'gbdt'
best_params['device'] = 'cpu'
best_params['n_estimators'] = 400

# Initialize list to store AUC scores
aucs = []
lgbm_models = []

# Iterate over folds
for fold in tqdm(range(N_FOLDS), total=N_FOLDS):
    # Get train and validation data
    train_data = metadata_df[metadata_df['fold'] != fold].reset_index(drop=True)
    val_data = metadata_df[metadata_df['fold'] == fold].reset_index(drop=True)

    # Define features and target
    features = train_data[train_cols]
    target = train_data['target']

    model = VotingClassifier([(f"lgb_{i}", lgb.LGBMClassifier(random_state=i, **best_params)) for i in range(3)], voting="soft")
    model.fit(features, target)
    # Predict probabilities for validation data
    preds_proba = model.predict_proba(val_data[train_cols])[:, 1]  # Probability of the positive class
    auc = pAUC(val_data['target'], preds_proba)
    print(f"Fold: {fold+1} - Partial AUC Score: {auc:.5f}")
    aucs.append(auc)
    lgbm_models.append(model)

print(f"Mean Partial AUC: {np.mean(aucs):.5f}")

  0%|          | 0/5 [00:00<?, ?it/s]

Fold: 1 - Partial AUC Score: 0.18219
Fold: 2 - Partial AUC Score: 0.17213
Fold: 3 - Partial AUC Score: 0.18086
Fold: 4 - Partial AUC Score: 0.17114
Fold: 5 - Partial AUC Score: 0.17409
Mean Partial AUC: 0.17608


# XGBoost

In [56]:
# Define objective function for XGBoost

def objective(trial):

    param = {
        "objective": "binary:logistic",
        "eval_metric": "auc",  # AUC will still be tracked as an eval metric
        "booster": "gbtree",
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.1, log=True),
        "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 100),
        "random_state": RANDOM_SEED
    }

    aucs = []

    for fold in range(N_FOLDS):
        train_data = metadata_df[metadata_df['fold'] != fold].reset_index(drop=True)
        val_data = metadata_df[metadata_df['fold'] == fold].reset_index(drop=True)

        features = train_data[train_cols]
        target = train_data['target']

        model = xgb.XGBClassifier(**param)
        # Train XGBoost model
        dtrain = xgb.DMatrix(features, label=target)
        dval = xgb.DMatrix(val_data[train_cols], label=val_data['target'])
        evals = [(dval, 'valid')]
        
        model = xgb.train(param, dtrain, evals=evals, num_boost_round=1000,
                          early_stopping_rounds=50, verbose_eval=False)
        
        preds = model.predict(dval)
        auc = pAUC(val_data['target'], preds)
        aucs.append(auc)
    
    return np.mean(aucs)

In [58]:
# Define study
study = optuna.create_study(direction="maximize", study_name='xgb_tuning')
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-11-12 16:12:02,622] A new study created in memory with name: xgb_tuning
[I 2024-11-12 16:12:19,402] Trial 0 finished with value: 0.1607741127845222 and parameters: {'learning_rate': 0.007327441405832055, 'lambda': 0.0014734484040870815, 'alpha': 1.1239934217046211e-07, 'max_depth': 4, 'subsample': 0.9583046087150832, 'colsample_bytree': 0.6469592511927842, 'min_child_weight': 22}. Best is trial 0 with value: 0.1607741127845222.
[I 2024-11-12 16:12:34,906] Trial 1 finished with value: 0.1552688550567906 and parameters: {'learning_rate': 0.006047666206177761, 'lambda': 3.79299802523574e-07, 'alpha': 0.0030505962888895412, 'max_depth': 3, 'subsample': 0.6884143670692255, 'colsample_bytree': 0.45439263973453725, 'min_child_weight': 75}. Best is trial 0 with value: 0.1607741127845222.
[I 2024-11-12 16:12:50,691] Trial 2 finished with value: 0.16781617708072938 and parameters: {'learning_rate': 0.00031293498531399783, 'lambda': 0.03299412197577287, 'alpha': 0.0009775070499208583, 'ma

Number of finished trials: 20
Best trial:
  Value: 0.17614381617293987
  Params: 
    learning_rate: 0.020258880560938636
    lambda: 3.290335620382773e-05
    alpha: 2.1147558243641616e-05
    max_depth: 2
    subsample: 0.8602868720210404
    colsample_bytree: 0.8826795748999703
    min_child_weight: 2


In [61]:
# Define best model
best_params = trial.params
best_params['objective'] = 'binary:logistic'
best_params['eval_metric'] = 'auc'
best_params['booster'] = 'gbtree'
best_params['n_estimators'] = 400

# Initialize list to store AUC scores and models for each fold
aucs = []
XGmodels = []

# Iterate over folds
for fold in tqdm(range(N_FOLDS)):
    # Get train and validation data for this fold
    train_data = metadata_df[metadata_df['fold'] != fold].reset_index(drop=True)
    val_data = metadata_df[metadata_df['fold'] == fold].reset_index(drop=True)

    # Define features and target
    X_train, y_train = train_data[train_cols], train_data['target']
    X_val, y_val = val_data[train_cols], val_data['target']

    model = VotingClassifier([(f"xgb_{i}", xgb.XGBClassifier(random_state=i, **best_params)) for i in range(3)], voting="soft")
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:, 1]
    # Calculate AUC score
    auc = pAUC(y_val, preds)
    print(f"Fold {fold} - AUC Score: {auc:.5f}")
    
    # Append results
    aucs.append(auc)
    XGmodels.append(model)

# Display average AUC across folds
print(f"Average AUC Score across folds: {np.mean(aucs):.5f}")

  0%|          | 0/5 [00:00<?, ?it/s]

Fold 0 - AUC Score: 0.17805
Fold 1 - AUC Score: 0.17137
Fold 2 - AUC Score: 0.17398
Fold 3 - AUC Score: 0.16612
Fold 4 - AUC Score: 0.17486
Average AUC Score across folds: 0.17287


In [84]:
len(lgbm_models), len(XGmodels)

(5, 5)